## Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm

from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score

#from nb21 import cumulative_gain, elast


## Cleaning and Encoding Data

In [ ]:
#df1 = pd.read_stata('workingpanel(a).dta')  ## test data

In [ ]:
##df1['industry'].unique()

In [ ]:
df = pd.read_stata('workingpanel.dta')

In [ ]:
df

In [ ]:
df.columns

In [ ]:
#df.union.replace(('Yes', 'No'), (1, 0), inplace=True)

In [ ]:
#df.sex.replace(('Male', 'Female'), (1, 0), inplace=True)

In [ ]:
df["sex"] = df["sex"].astype('category')
df["sex_cat"] = df["sex"].cat.codes
#df.head()

In [ ]:
df.mediumskill.replace((np.nan, 1), (0, 1), inplace=True)

In [ ]:
df["mediumskill"] = df["mediumskill"].astype('category')
df["mediumskill_cat"] = df["mediumskill"].cat.codes
#df.head()

In [ ]:
df["healthlim"] = df["healthlim"].astype('category')
df["healthlim_cat"] = df["healthlim"].cat.codes
#df.head()

In [ ]:
df["private"] = df["private"].astype('category')
df["private_cat"] = df["private"].cat.codes
#df.head()

In [ ]:
df["size"] = df["size"].astype('category')
df["size_cat"] = df["size"].cat.codes
#df.head()

In [ ]:
df["occup"] = df["occup"].astype('category')
df["occup_cat"] = df["occup"].cat.codes
#df.head()

In [ ]:
df["industry"] = df["industry"].astype('category')
df["industry_cat"] = df["industry"].cat.codes
df.head()

In [ ]:
df.describe()

In [ ]:
df = df[['cons','Dunion','lhw','expr','myedu','mediumskill_cat','healthlim_cat','sex_cat','private_cat','size_cat','occup_cat','industry_cat']]

In [ ]:
df.describe()


## Covariates Balance

In [ ]:
from sklearn.manifold import TSNE
import plotly.express as px

features = df[['expr','myedu','mediumskill_cat','healthlim_cat','sex_cat','private_cat','size_cat','occup_cat','industry_cat']]

tsne = TSNE(n_components=2, random_state=0)
projections = tsne.fit_transform(features)

fig = px.scatter(
    projections, x=0, y=1,
    color=df.Dunion.replace((0, 1), ("Control", "Treated"), inplace=False), template = "simple_white",labels={'color': 'Dunion'},color_discrete_map={'Treated':'#FFA500','Control':'#1E90FF'}
        )

fig.update_yaxes(visible=False, showticklabels=False)
fig.update_xaxes(visible=False, showticklabels=False)
fig.show()



In [ ]:
## Balance

fig, axs = plt.subplots(3, 3, figsize=(18, 10))
fig.suptitle('Marginals Distributions for Covariates X',fontsize = 32)
sns.set(font_scale=1.8)
plt.tight_layout()
sns.histplot(df, x='expr', hue="Dunion", stat="proportion",multiple='dodge',common_norm=False, legend =False, ax=axs[0,0]).set(xlabel='Experience', ylabel='Marginals') 
sns.histplot(df, x='myedu', hue="Dunion", stat="proportion",multiple='dodge',common_norm=False, legend =False, ax=axs[0,1]).set(xlabel='Education', ylabel='Marginals')
sns.histplot(df, x='mediumskill_cat', hue="Dunion", stat="proportion",multiple='dodge',common_norm=False, legend =False, ax=axs[0,2]).set(xlabel='Medium Skills', ylabel='Marginals') 
sns.histplot(df, x='healthlim_cat', hue="Dunion", stat="proportion",multiple='dodge',common_norm=False, legend =False,ax=axs[1,0]).set(xlabel='Health Limitations', ylabel='Marginals')
sns.histplot(df, x='sex_cat', hue="Dunion", stat="proportion",multiple='dodge',common_norm=False, legend =False,ax=axs[1,1]).set(xlabel='Gender', ylabel='Marginals')
sns.histplot(df, x='private_cat', hue="Dunion", stat="proportion",multiple='dodge',common_norm=False, legend =False, ax=axs[1,2]).set(xlabel='Private Sector Worker', ylabel='Marginals')
sns.histplot(df, x='size_cat', hue="Dunion", stat="proportion",multiple='dodge',common_norm=False, legend =False,ax=axs[2,0]).set(xlabel='Firm Size', ylabel='Marginals')
sns.histplot(df, x='occup_cat', hue="Dunion", stat="proportion",multiple='dodge',common_norm=False, legend =False,ax=axs[2,1]).set(xlabel='Occupations', ylabel='Marginals')
sns.histplot(df, x='industry_cat', hue="Dunion", stat="proportion",multiple='dodge',common_norm=False, legend =False,ax=axs[2,2]).set(xlabel='Industries', ylabel='Marginals')

plt.legend(labels=["Treated","Control"], bbox_to_anchor=(1,5))
plt.savefig('covariatemargins_v2.pdf')


## Split test and train data

In [ ]:
y = "lhw"
T = "Dunion"
X = ['expr','myedu','mediumskill_cat','healthlim_cat','sex_cat','private_cat','size_cat','occup_cat','industry_cat'] ##covars that I consider for CATE 

In [ ]:
X

In [ ]:
from sklearn.model_selection import train_test_split

np.random.seed(123)
train,test = train_test_split(df, test_size=0.3)
print(train.shape, test.shape)

## S-Learner 

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from econml.sklearn_extensions.model_selection import GridSearchCVList
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.base import clone
from econml.sklearn_extensions.linear_model import WeightedLasso
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

np.random.seed(123)
s_learner = RandomForestRegressor(n_estimators=100,min_samples_leaf=10, random_state=123) ##Gridsearch model
s_learner.fit(train[X+[T]], train[y])

In [ ]:
##Treatment Effect
treatment_effect1 = train.query('Dunion==1')['lhw'].mean(axis = 0)
treatment_effect0 = train.query('Dunion==0')['lhw'].mean(axis = 0)

naive_te = treatment_effect1 - treatment_effect0 ##naive estimator


In [ ]:
naive_te

In [ ]:
s_learner_cate_train = (s_learner.predict(train[X].assign(**{T: 1})) -
                        s_learner.predict(train[X].assign(**{T: 0})))

s_learner_cate_test = test.assign(
    cate=(s_learner.predict(test[X].assign(**{T: 1})) - # predict under treatment
          s_learner.predict(test[X].assign(**{T: 0}))) # predict under control
)

In [ ]:
s_learner_cate_test[['cate']] ##CATE

In [ ]:
baseline = s_learner_cate_test[['cate']].mean(axis = 0) #ATE

In [ ]:
baseline[0] ## este es el mejor modelo!

In [ ]:
s_learner.score(test[X+[T]],test[y])  #R2

In [ ]:
mse = mean_squared_error(test[y],s_learner.predict(test[X+[T]])) #we can set any metric av in sklearn

In [ ]:
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

In [ ]:
rsqua = r2_score(test[y],s_learner.predict(test[X+[T]]))

In [ ]:
rsqua ## R2 just checking... 

In [ ]:
slearner_cate = s_learner_cate_test[['cate']].values

In [ ]:
import scipy.stats as st ### INTERVALO DE CONFIANZA ATE hay q ver para los de econML
st.norm.interval(confidence=0.99, 
                 loc=np.mean(slearner_cate),
                 scale=st.sem(slearner_cate))

## X-Learner

In [ ]:
from sklearn.linear_model import LogisticRegression

np.random.seed(123)

# first stage models (this can be changed!) 
m0 = RandomForestRegressor(n_estimators=100,min_samples_leaf=10, random_state=123)
m1 = RandomForestRegressor(n_estimators=100,min_samples_leaf=10, random_state=123)

# propensity score model
g = RandomForestClassifier(n_estimators=100, max_depth=6) 

m0.fit(train.query(f"{T}==0")[X], train.query(f"{T}==0")[y])
m1.fit(train.query(f"{T}==1")[X], train.query(f"{T}==1")[y])
                       
g.fit(train[X], train[T]);

In [ ]:
d_train = np.where(train[T]==0,
                   m1.predict(train[X]) - train[y],
                   train[y] - m0.predict(train[X]))

# second stage
mx0 = RandomForestRegressor(n_estimators=100,min_samples_leaf=10, random_state=123)
mx1 = RandomForestRegressor(n_estimators=100,min_samples_leaf=10, random_state=123)

mx0.fit(train.query(f"{T}==0")[X], d_train[train[T]==0])
mx1.fit(train.query(f"{T}==1")[X], d_train[train[T]==1]);

In [ ]:
def ps_predict(df, t): 
    return g.predict_proba(df[X])[:, t]
    
    
x_cate_train = (ps_predict(train,0)*mx0.predict(train[X]) +
                ps_predict(train,1)*mx1.predict(train[X]))

x_cate_test = test.assign(cate=(ps_predict(test,0)*mx0.predict(test[X]) +
                                ps_predict(test,1)*mx1.predict(test[X])))

In [ ]:
x_cate_test[['cate']] ##CATE

In [ ]:
#baseline2 = x_cate_test[['cate']].mean(axis = 0) #ATE

In [ ]:
#baseline2

In [ ]:
m0.score(test[X],test[y])  #r2

## T-Learner

In [ ]:
np.random.seed(123)

m0 = RandomForestRegressor(n_estimators=100,min_samples_leaf=10, random_state=123)
m1 = RandomForestRegressor(n_estimators=100,min_samples_leaf=10, random_state=123)

m0.fit(train.query(f"{T}==0")[X], train.query(f"{T}==0")[y])
m1.fit(train.query(f"{T}==1")[X], train.query(f"{T}==1")[y])

# estimate the CATE
t_learner_cate_train = m1.predict(train[X]) - m0.predict(train[X])
t_learner_cate_test = test.assign(cate=m1.predict(test[X]) - m0.predict(test[X]))

In [ ]:
t_learner_cate_test[['cate']] ##CATE

In [ ]:
#baseline3 = t_learner_cate_test[['cate']].mean(axis = 0) #ATE

In [ ]:
#baseline3

In [ ]:
m0.score(test[X],test[y])  #r2

### EconML library

In [ ]:
from econml.metalearners import TLearner, SLearner, XLearner, DomainAdaptationLearner

In [ ]:
df.head()

In [ ]:
np.random.seed(123)
train,test = train_test_split(df, test_size=0.3)
print(train.shape, test.shape)

In [ ]:
Y = train["lhw"].values
T = train["Dunion"].values
X = train[['expr','myedu','mediumskill_cat','healthlim_cat','sex_cat','private_cat','size_cat','occup_cat','industry_cat']].values ## ADD MORE VARS
X_test = test[['expr','myedu','mediumskill_cat','healthlim_cat','sex_cat','private_cat','size_cat','occup_cat','industry_cat']].values
Y_test = test["lhw"].values
T_test = test["Dunion"].values

In [ ]:
from econml.sklearn_extensions.model_selection import GridSearchCVList
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.base import clone
from econml.sklearn_extensions.linear_model import WeightedLasso

##CHOOSING THE BEST HYPERPARAMETERS FOR FIRST STAGE MODELS!

def first_stage_reg():  ##grid search example
    return GridSearchCVList([Lasso(),
                             RandomForestRegressor(n_estimators=100, random_state=123),
                             GradientBoostingRegressor()
                            ],
                             param_grid_list=[{'alpha': [.001, .01, .1, 1, 10]},
                                               {'max_depth': [3, None],
                                               'min_samples_leaf': [10, 50]},
                                              {"max_depth": [3, 5, None], "n_estimators": [50, 100, 200]}
                                             ],
                             cv=5,
                             scoring='r2')

In [ ]:
model_y = clone(first_stage_reg().fit(X, Y).best_estimator_)
model_y

In [ ]:
# Instantiate S learner
overall_model = model_y
S_learner = SLearner(overall_model=overall_model)
# Train S_learner
S_learner.fit(Y, T, X=X)
# Estimate treatment effects on test data
S_te = S_learner.effect(X_test) #CATE

In [ ]:
ateS = np.mean(S_te)

In [ ]:
ateS ##checking values... OK!

In [ ]:
S_te #treatment effects

In [ ]:
S_lear = S_learner.fit(Y, T, X=X)

In [ ]:
import scipy.stats as st ### INTERVALO DE CONFIANZA 
st.norm.interval(confidence=0.99, 
                 loc=np.mean(S_te),
                 scale=st.sem(S_te))

In [ ]:
import shap ##SHAP interpreter; mutted because took so long
slear = S_learner.fit(Y, T, X=X)
shap_values = slear.shap_values(X[:])




In [ ]:
shap_values

In [ ]:
import matplotlib.pyplot as plt
shap.plots.beeswarm(shap_values['Y0']['T0_1.0'],show=False)
plt.savefig("shap_summary_v2.pdf",dpi=700) #.png,.pdf will also support here


In [ ]:
# Instantiate X learner
models = model_y
propensity_model = RandomForestClassifier(n_estimators=100, max_depth=6)
X_learner = XLearner(models=models, propensity_model=propensity_model)
# Train X_learner
X_learner.fit(Y, T, X=X)
# Estimate treatment effects on test data
X_te = X_learner.effect(X_test)

In [ ]:
X_te
ateX = np.mean(X_te)
ateX

In [ ]:
st.norm.interval(confidence=0.99, 
                 loc=np.mean(X_te),
                 scale=st.sem(X_te))

In [ ]:
# Instantiate T learner
models = model_y
T_learner = TLearner(models=models)
# Train T_learner
T_learner.fit(Y, T, X=X)
# Estimate treatment effects on test data
T_te = T_learner.effect(X_test)

In [ ]:
T_te

In [ ]:
ateT = np.mean(T_te)

In [ ]:
ateT

In [ ]:
st.norm.interval(confidence=0.99, 
                 loc=np.mean(T_te),
                 scale=st.sem(T_te))

In [ ]:
from matplotlib.ticker import NullFormatter, FixedLocator
plt.figure(figsize=(15,15))

#1
plt.subplot2grid((5,2), (0,0))
plt.axhline(naive_te, color = 'black', linestyle = '--', label = 'Naive Estimator')
plt.scatter(X_test[:,0],T_te, label='T-learner',color='#FFA500') 
plt.scatter(X_test[:,0],X_te, label='X-learner',color='#D6D4D7') 
plt.scatter(X_test[:,0],S_te, label='S-learner',color='#1E90FF')
plt.xlabel('$x_0$')
plt.ylabel('CATE')
#plt.legend() ##check this here and in the cates one
plt.title('Experience')

#2
plt.subplot2grid((5,2), (0,1))
plt.axhline(naive_te, color = 'black', linestyle = '--', label = 'Naive Estimator')
plt.scatter(X_test[:,1],T_te, label='T-learner',color='#FFA500') 
plt.scatter(X_test[:,1],X_te, label='X-learner',color='#D6D4D7') 
plt.scatter(X_test[:,1],S_te, label='S-learner',color='#1E90FF') 
plt.xlabel('$x_1$')
plt.ylabel('CATE')
#plt.legend()
plt.title('Education Level')

#3
plt.subplot2grid((5,2), (1,0))
plt.axhline(naive_te, color = 'black', linestyle = '--', label = 'Naive Estimator')
plt.scatter(X_test[:,2],T_te, label='T-learner',color='#FFA500') 
plt.scatter(X_test[:,2],X_te, label='X-learner',color='#D6D4D7') 
plt.scatter(X_test[:,2],S_te, label='S-learner',color='#1E90FF') 
plt.xlabel('$x_2$')
plt.ylabel('CATE')
#plt.legend()
plt.title('Medium Skill')

#4
plt.subplot2grid((5,2), (1,1))
plt.axhline(naive_te, color = 'black', linestyle = '--', label = 'Naive Estimator')
plt.scatter(X_test[:,3],T_te, label='T-learner',color='#FFA500') 
plt.scatter(X_test[:,3],X_te, label='X-learner',color='#D6D4D7') 
plt.scatter(X_test[:,3],S_te, label='S-learner',color='#1E90FF') 
plt.xlabel('$x_3$')
plt.ylabel('CATE')
#plt.legend()
plt.title('Health Limitations')

#5
plt.subplot2grid((5,2), (2,0))
plt.axhline(naive_te, color = 'black', linestyle = '--', label = 'Naive Estimator')
plt.scatter(X_test[:,4],T_te, label='T-learner',color='#FFA500') 
plt.scatter(X_test[:,4],X_te, label='X-learner',color='#D6D4D7') 
plt.scatter(X_test[:,4],S_te, label='S-learner',color='#1E90FF') 
plt.xlabel('$x_4$')
plt.ylabel('CATE')
#plt.legend()
plt.title('Gender')


#6
plt.subplot2grid((5,2), (2,1))
plt.axhline(naive_te, color = 'black', linestyle = '--', label = 'Naive Estimator')
plt.scatter(X_test[:,6],T_te, label='T-learner',color='#FFA500') 
plt.scatter(X_test[:,6],X_te, label='X-learner',color='#D6D4D7') 
plt.scatter(X_test[:,6],S_te, label='S-learner',color='#1E90FF')
plt.xlabel('$x_6$')
plt.ylabel('CATE')
#plt.legend()
plt.title('Firm Size')


#7
plt.subplot2grid((5,2), (3,0))
plt.axhline(naive_te, color = 'black', linestyle = '--', label = 'Naive Estimator')
plt.scatter(X_test[:,7],T_te, label='T-learner',color='#FFA500') 
plt.scatter(X_test[:,7],X_te, label='X-learner',color='#D6D4D7') 
plt.scatter(X_test[:,7],S_te, label='S-learner',color='#1E90FF') 
plt.xlabel('$x_7$')
plt.ylabel('CATE')
#plt.legend()
plt.title('Occupations')

#8
plt.subplot2grid((5,2), (3,1))
plt.axhline(naive_te, color = 'black', linestyle = '--', label = 'Naive Estimator')
plt.scatter(X_test[:,8],T_te, label='T-learner',color='#FFA500') 
plt.scatter(X_test[:,8],X_te, label='X-learner',color='#D6D4D7')
plt.scatter(X_test[:,8],S_te, label='S-learner',color='#1E90FF') 
plt.xlabel('$x_8$')
plt.ylabel('CATE')
#plt.legend()
plt.title('Industries')



plt.gca().yaxis.set_minor_formatter(NullFormatter())
plt.subplots_adjust(top=0.92, bottom=0.08, left=0.20, right=0.95, hspace=0.7,
                    wspace=0.35)
plt.legend(bbox_to_anchor=(2,5), loc="upper right")
plt.savefig('CATEvsCovs_vert_v2.pdf')
plt.show()






In [ ]:
## same chart than above but for the presentation
from matplotlib.ticker import NullFormatter, FixedLocator
plt.figure(figsize=(10, 5))

#1
plt.subplot2grid((2,3), (0,0))
plt.axhline(naive_te, color = 'black', linestyle = '--', label = 'Naive Estimator')
plt.scatter(X_test[:,0],T_te, label='T-learner',color='#FFA500') 
plt.scatter(X_test[:,0],X_te, label='X-learner',color='#D6D4D7') 
plt.scatter(X_test[:,0],S_te, label='S-learner',color='#1E90FF')
plt.xlabel('$x_0$')
plt.ylabel('CATE')
#plt.legend() ##check this here and in the cates one
plt.title('Experience')


#3
plt.subplot2grid((2,3), (0,2))
plt.axhline(naive_te, color = 'black', linestyle = '--', label = 'Naive Estimator')
plt.scatter(X_test[:,1],T_te, label='T-learner',color='#FFA500') 
plt.scatter(X_test[:,1],X_te, label='X-learner',color='#D6D4D7') 
plt.scatter(X_test[:,1],S_te, label='S-learner',color='#1E90FF') 
plt.xlabel('$x_1$')
plt.ylabel('CATE')
#plt.legend()
plt.title('Education Level')

#5
plt.subplot2grid((2,3), (0,1))
plt.axhline(naive_te, color = 'black', linestyle = '--', label = 'Naive Estimator')
plt.scatter(X_test[:,5],T_te, label='T-learner',color='#FFA500') 
plt.scatter(X_test[:,5],X_te, label='X-learner',color='#D6D4D7') 
plt.scatter(X_test[:,5],S_te, label='S-learner',color='#1E90FF') 
plt.xlabel('$x_5$')
plt.ylabel('CATE')
#plt.legend()
plt.title('Health Limitations')


#6
plt.subplot2grid((2,3), (1,0))
plt.axhline(naive_te, color = 'black', linestyle = '--', label = 'Naive Estimator')
plt.scatter(X_test[:,6],T_te, label='T-learner',color='#FFA500') 
plt.scatter(X_test[:,6],X_te, label='X-learner',color='#D6D4D7') 
plt.scatter(X_test[:,6],S_te, label='S-learner',color='#1E90FF') 
plt.xlabel('$x_6$')
plt.ylabel('CATE')
#plt.legend()
plt.title('Firm Size')


#7
plt.subplot2grid((2,3), (1,1))
plt.axhline(naive_te, color = 'black', linestyle = '--', label = 'Naive Estimator')
plt.scatter(X_test[:,7],T_te, label='T-learner',color='#FFA500') 
plt.scatter(X_test[:,7],X_te, label='X-learner',color='#D6D4D7') 
plt.scatter(X_test[:,7],S_te, label='S-learner',color='#1E90FF') 
plt.xlabel('$x_7$')
plt.ylabel('CATE')
#plt.legend()
plt.title('Occupations')

#8
plt.subplot2grid((2,3), (1,2))
plt.axhline(naive_te, color = 'black', linestyle = '--', label = 'Naive Estimator')
plt.scatter(X_test[:,8],T_te, label='T-learner',color='#FFA500') 
plt.scatter(X_test[:,8],X_te, label='X-learner',color='#D6D4D7')
plt.scatter(X_test[:,8],S_te, label='S-learner',color='#1E90FF') 
plt.xlabel('$x_8$')
plt.ylabel('CATE')
#plt.legend()
plt.title('Industries')



plt.gca().yaxis.set_minor_formatter(NullFormatter())
plt.subplots_adjust(top=0.92, bottom=0.08, left=0.10, right=0.95, hspace=0.7,
                    wspace=0.45)
plt.legend(bbox_to_anchor=(2,2.7), loc="upper right")
plt.savefig('CATEvsCovs_hor_v2.pdf')
plt.show()

## CAUSAL FOREST (TO SELECT FEATURE IMPORTANCE) 

In [ ]:
from econml.grf import CausalForest, CausalIVForest, RegressionForest
from econml.dml import CausalForestDML
import scipy.special
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

In [ ]:
df

In [ ]:
np.random.seed(123)
train,test = train_test_split(df, test_size=0.3)
print(train.shape, test.shape)

Y = train["lhw"].values
T = train["Dunion"].values
X = train[['expr','myedu','mediumskill_cat','healthlim_cat','sex_cat','private_cat','size_cat','occup_cat','industry_cat']].values
X_test = test[['expr','myedu','mediumskill_cat','healthlim_cat','sex_cat','private_cat','size_cat','occup_cat','industry_cat']].values
Y_test = test["lhw"].values
true_te = lambda X: np.hstack([(X[:, [0]]>0) * X[:, [0]],
                               np.ones((X.shape[0], 1 - 1))*np.arange(1, 1).reshape(1, -1)])


In [ ]:
est = CausalForest(criterion='het', n_estimators=400, min_samples_leaf=5, max_depth=None,
                   min_var_fraction_leaf=None, min_var_leaf_on_val=True,
                   min_impurity_decrease = 0.0, max_samples=0.45, min_balancedness_tol=.45,
                   warm_start=False, inference=True, fit_intercept=True, subforest_size=4,
                   honest=True, verbose=0, n_jobs=-1, random_state=1235)

In [ ]:
est.fit(X,T, Y)

In [ ]:
plt.figure(figsize=(20, 10))
plot_tree(est[0], impurity=True, max_depth=3)
plt.savefig('CF_tree_v2.pdf')
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(est.feature_importances(max_depth=4, depth_decay_exponent=2.0))
plt.savefig('feature_importance_v2.pdf')
plt.show()

In [ ]:
## individual models

In [ ]:
np.random.seed(123)
train,test = train_test_split(df, test_size=0.3)
print(train.shape, test.shape)

Y = train["lhw"].values
T = train["Dunion"].values
X8 = train[["industry_cat"]].values
X8_test = test[["industry_cat"]].values
Y_test = test["lhw"].values
true_te = lambda X: np.hstack([(X[:, [0]]>0) * X[:, [0]],
                               np.ones((X.shape[0], 1 - 1))*np.arange(1, 1).reshape(1, -1)])

In [ ]:
# Instantiate S learner
overall_model = model_y
S_learner = SLearner(overall_model=overall_model)
# Train S_learner
S_learner.fit(Y, T, X=X8)
# Estimate treatment effects on test data
S_te_8 = S_learner.effect(X8_test) #CATE

In [ ]:
ateS = np.mean(S_te_8)
ateS

In [ ]:
#chart
#plt.figure(figsize=(8, 5))
#plt.axhline(baseline[0], color = 'r', linestyle = '-', label = 'ATE')
#plt.scatter(X9_test[:,0],S_te_9, label='S-learner',color='#1E90FF') ##CHECK THIS
#plt.xlabel('$x_0$')
#plt.ylabel('CATE')
#plt.legend()
#plt.show()

In [ ]:

np.random.seed(123)
train,test = train_test_split(df, test_size=0.3)
print(train.shape, test.shape)

Y = train["lhw"].values
T = train["Dunion"].values
X0 = train[["expr"]].values
X0_test = test[["expr"]].values
Y_test = test["lhw"].values
true_te = lambda X: np.hstack([(X[:, [0]]>0) * X[:, [0]],
                               np.ones((X.shape[0], 1 - 1))*np.arange(1, 1).reshape(1, -1)])



In [ ]:
# Instantiate S learner
overall_model = model_y
S_learner = SLearner(overall_model=overall_model)
# Train S_learner
S_learner.fit(Y, T, X=X0)
# Estimate treatment effects on test data
S_te_0 = S_learner.effect(X0_test) #CATE

In [ ]:
ateS = np.mean(S_te_0)
ateS

In [ ]:
##plt.figure(figsize=(8, 5))
##plt.axhline(baseline[0], color = 'r', linestyle = '-', label = 'ATE')
##plt.scatter(X0_test[:,0],S_te_0, label='S-learner',color='#1E90FF') ##CHECK THIS
##plt.xlabel('$x_0$')
##plt.ylabel('CATE')
##plt.legend()
##plt.show()

In [ ]:

np.random.seed(123)
train,test = train_test_split(df, test_size=0.3)
print(train.shape, test.shape)

Y = train["lhw"].values
T = train["Dunion"].values
X5 = train[["healthlim_cat"]].values
X5_test = test[["healthlim_cat"]].values
Y_test = test["lhw"].values
true_te = lambda X: np.hstack([(X[:, [0]]>0) * X[:, [0]],
                               np.ones((X.shape[0], 1 - 1))*np.arange(1, 1).reshape(1, -1)])


In [ ]:
# Instantiate S learner
overall_model = model_y
S_learner = SLearner(overall_model=overall_model)
# Train S_learner
S_learner.fit(Y, T, X=X5)
# Estimate treatment effects on test data
S_te_5 = S_learner.effect(X5_test) #CATE

In [ ]:
ateS = np.mean(S_te_5)
ateS

In [ ]:
##plt.figure(figsize=(8, 5))
##plt.axhline(baseline[0], color = 'r', linestyle = '-', label = 'ATE')
##plt.scatter(X1_test[:,0],S_te_1, label='S-learner',color='#1E90FF') ##CHECK THIS
##plt.xlabel('$x_0$')
##plt.ylabel('CATE')
##plt.legend()
##plt.show()

In [ ]:
np.random.seed(123)
train,test = train_test_split(df, test_size=0.3)
print(train.shape, test.shape)

Y = train["lhw"].values
T = train["Dunion"].values
X7 = train[["occup_cat"]].values
X7_test = test[["occup_cat"]].values
Y_test = test["lhw"].values
true_te = lambda X: np.hstack([(X[:, [0]]>0) * X[:, [0]],
                               np.ones((X.shape[0], 1 - 1))*np.arange(1, 1).reshape(1, -1)])

In [ ]:
# Instantiate S learner
overall_model = model_y
S_learner = SLearner(overall_model=overall_model)
# Train S_learner
S_learner.fit(Y, T, X=X7)
# Estimate treatment effects on test data
S_te_7 = S_learner.effect(X7_test) #CATE

In [ ]:
ateS = np.mean(S_te_7)
ateS

In [ ]:
##plt.figure(figsize=(8, 5))
##plt.axhline(baseline[0], color = 'r', linestyle = '-', label = 'ATE')
##plt.scatter(X8_test[:,0],S_te_8, label='S-learner',color='#1E90FF') ##CHECK THIS
##plt.xlabel('$x_0$')
##plt.ylabel('CATE')
##plt.legend()
##plt.show()

In [ ]:
np.random.seed(123)
train,test = train_test_split(df, test_size=0.3)
print(train.shape, test.shape)

Y = train["lhw"].values
T = train["Dunion"].values
X6 = train[["size_cat"]].values
X6_test = test[["size_cat"]].values
Y_test = test["lhw"].values
true_te = lambda X: np.hstack([(X[:, [0]]>0) * X[:, [0]],
                               np.ones((X.shape[0], 1 - 1))*np.arange(1, 1).reshape(1, -1)])

In [ ]:
# Instantiate S learner
overall_model = model_y
S_learner = SLearner(overall_model=overall_model)
# Train S_learner
S_learner.fit(Y, T, X=X6)
# Estimate treatment effects on test data
S_te_6 = S_learner.effect(X6_test) #CATE

In [ ]:
ateS = np.mean(S_te_6)
ateS

In [ ]:
##plt.figure(figsize=(8, 5))
##plt.axhline(baseline[0], color = 'r', linestyle = '-', label = 'ATE')
##plt.scatter(X7_test[:,0],S_te_7, label='S-learner',color='#1E90FF') ##CHECK THIS
##plt.xlabel('$x_0$')
##plt.ylabel('CATE')
##plt.legend()
##plt.show()

In [ ]:
np.random.seed(123)
train,test = train_test_split(df, test_size=0.3)
print(train.shape, test.shape)

Y = train["lhw"].values
T = train["Dunion"].values
X4 = train[["sex_cat"]].values
X4_test = test[["sex_cat"]].values
Y_test = test["lhw"].values
true_te = lambda X: np.hstack([(X[:, [0]]>0) * X[:, [0]],
                               np.ones((X.shape[0], 1 - 1))*np.arange(1, 1).reshape(1, -1)])

In [ ]:
# Instantiate S learner
overall_model = model_y
S_learner = SLearner(overall_model=overall_model)
# Train S_learner
S_learner.fit(Y, T, X=X4)
# Estimate treatment effects on test data
S_te_4 = S_learner.effect(X4_test) #CATE

In [ ]:
ateS = np.mean(S_te_4)
ateS

In [ ]:
##plt.figure(figsize=(8, 5))
##plt.axhline(baseline[0], color = 'r', linestyle = '-', label = 'ATE')
##plt.scatter(X5_test[:,0],S_te_5, label='S-learner',color='#1E90FF') ##CHECK THIS
##plt.xlabel('$x_0$')
##plt.ylabel('CATE')
##plt.legend()
##plt.show()

In [ ]:
from matplotlib import cm
from matplotlib.ticker import AutoMinorLocator
from matplotlib.image import NonUniformImage
from matplotlib.ticker import NullFormatter  


plt.figure(figsize=(10, 10))

#1
plt.subplot2grid((3,2), (0,0))
plt.axhline(baseline[0], color = 'r', linestyle = '--', label = 'ATE')
plt.scatter(X8_test[:,0],S_te_8, label='S-learner',color='#1E90FF') ##CHECK THIS
plt.xlabel('$x_8$')
plt.ylabel('CATE')
##plt.legend()
plt.title('Industries')

#2
plt.subplot2grid((3,2), (0,1))
plt.axhline(baseline[0], color = 'r', linestyle = '--', label = 'ATE')
plt.scatter(X0_test[:,0],S_te_0, label='S-learner',color='#1E90FF') ##CHECK THIS
plt.xlabel('$x_0$')
plt.ylabel('CATE')
##plt.legend()
plt.title('Experience')

#3
plt.subplot2grid((3,2), (1,0))
plt.axhline(baseline[0], color = 'r', linestyle = '--', label = 'ATE')
plt.scatter(X5_test[:,0],S_te_5, label='S-learner',color='#1E90FF') ##CHECK THIS
plt.xlabel('$x_5$')
plt.ylabel('CATE')
##plt.legend()
plt.title('Health Limitations')


#4
plt.subplot2grid((3,2), (1,1))
plt.axhline(baseline[0], color = 'r', linestyle = '--', label = 'ATE')
plt.scatter(X7_test[:,0],S_te_7, label='S-learner',color='#1E90FF') ##CHECK THIS
plt.xlabel('$x_7$')
plt.ylabel('CATE')
##plt.legend()
plt.title('Occupations')

#5
plt.subplot2grid((3,2), (2,0))
plt.axhline(baseline[0], color = 'r', linestyle = '--', label = 'ATE')
plt.scatter(X6_test[:,0],S_te_6, label='S-learner',color='#1E90FF') ##CHECK THIS
plt.xlabel('$x_6$')
plt.ylabel('CATE')
##plt.legend()
plt.title('Firm Size')

#6
plt.subplot2grid((3,2), (2,1))
plt.axhline(baseline[0], color = 'r', linestyle = '--', label = 'ATE')
plt.scatter(X4_test[:,0],S_te_4, label='S-learner',color='#1E90FF') ##CHECK THIS
plt.xlabel('$x_4$')
plt.ylabel('CATE')
plt.legend()
plt.title('Gender')


plt.gca().yaxis.set_minor_formatter(NullFormatter())
plt.subplots_adjust(top=0.92, bottom=0.08, left=0.20, right=0.95, hspace=0.65,
                    wspace=0.35)
plt.legend(bbox_to_anchor=(2,4), loc="upper right")
plt.savefig('het_cates_v2.pdf')
plt.show()